In [33]:
import pandas as pd

In [34]:
pitch_df = pd.read_csv('./Datasets/pitcher_stats.csv')
lineups = pd.read_csv('./Datasets/Lineups_2021_07_02.csv')
bat_df = pd.read_csv('./Datasets/batter_stats.csv')

In [35]:
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')

In [36]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [37]:
lineups.columns = lineups.columns.str.replace(' player name', 'Name')
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'
pitch_df.drop(456, inplace = True)

In [38]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [39]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [40]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-40-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-40-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [41]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-41-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [42]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [43]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [44]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [45]:
sals = pd.read_csv('./DKSalaries.csv')

In [46]:
sals.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,SP,Trevor Williams (18269392),Trevor Williams,18269392,P,12700,CHC@CIN 07/02/2021 07:10PM ET,CHC,11.30
1,SP,Tyler Glasnow (18269393),Tyler Glasnow,18269393,P,12600,TB@TOR 07/02/2021 07:07PM ET,TB,25.91
2,RP,Shane McClanahan (18269394),Shane McClanahan,18269394,P,12400,TB@TOR 07/02/2021 07:07PM ET,TB,14.87
3,SP,Kyle Hendricks (18269395),Kyle Hendricks,18269395,P,12200,CHC@CIN 07/02/2021 07:10PM ET,CHC,15.28
4,SP,Yonny Chirinos (18269396),Yonny Chirinos,18269396,P,11900,TB@TOR 07/02/2021 07:07PM ET,TB,0.00


In [47]:
sals.rename(columns = {'Salary':'DK sal'}, inplace = True)

In [48]:
pitchers = pd.merge(pitchers, sals[['Name', 'DK sal']])

In [49]:
p_feat = ['Name', 'num_L_bats', 'num_R_bats',
       'is_nl', 'k_9_l', 'bb_9_l', 'k_bb_l', 'hr_9_l', 'k_pct_l', 'bb_pct_l',
       'k_bb_pct_l', 'bat_avg_l', 'whip_l', 'babip_l', 'lob_pct_l', 'fip_l',
       'xfip_l', 'k_9_r', 'bb_9_r', 'k_bb_r', 'hr_9_r', 'k_pct_r', 'bb_pct_r',
       'k_bb_pct_r', 'bat_avg_r', 'whip_r', 'babip_r', 'lob_pct_r', 'fip_r',
       'xfip_r', 'k_9_l_tot', 'bb_9_l_tot', 'k_bb_l_tot', 'hr_9_l_tot',
       'k_pct_l_tot', 'bb_pct_l_tot', 'k_bb_pct_l_tot', 'bat_avg_l_tot',
       'whip_l_tot', 'babip_l_tot', 'lob_pct_l_tot', 'fip_l_tot', 'xfip_l_tot',
       'k_9_r_tot', 'bb_9_r_tot', 'k_bb_r_tot', 'hr_9_r_tot', 'k_pct_r_tot',
       'bb_pct_r_tot', 'k_bb_pct_r_tot', 'bat_avg_r_tot', 'whip_r_tot',
       'babip_r_tot', 'lob_pct_r_tot', 'fip_r_tot', 'xfip_r_tot', 'DK sal']
final_pitch = pitchers[p_feat]

In [50]:
final_pitch.to_csv('./merged_data/for_preds_nn.csv', index = False)